In [13]:

#Importing all the relevant library
%matplotlib inline
import h5py, os
#from functions import transforms as T
#from functions.subsample import MaskFunc
from scipy.io import loadmat
from torch.utils.data import DataLoader
import numpy as np
import torch
from matplotlib import pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as T
import glob
#from functions import transforms as T 
#from functions.subsample import MaskFunc
from PIL import Image
import random
from numpy.fft import fftshift, ifftshift, fftn, ifftn
import cmath

import os
import random
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import tensorflow as tf
from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape
from keras.layers import Activation, BatchNormalization, LeakyReLU, Dropout, MaxPool2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam, RMSprop, Adagrad, SGD
from keras.utils import plot_model
from skimage.io import imread
from PIL import Image, ImageDraw
import cv2 as cv
import math
import h5py
from keras.models import load_model
import json


In [14]:
import torchvision.transforms as transforms

In [15]:
class MRIDataset(DataLoader):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        subject_id = self.data_list[idx]

        return get_epoch_batch(subject_id)

In [16]:
import random
import numpy as np
from numpy.fft import fftshift, ifftshift, fftn, ifftn
import cmath
def noise_and_kspace(image):
    #change to k-space
    img_fft = fftshift(fftn(image))
    size_img = img_fft.shape
     #np.random.uniform, np.random.normal
    std = np.random.normal(0.000, 0.005) * np.amax(img_fft)
    noise = fftshift(std * np.random.standard_normal(size_img) + std * 1j * np.random.standard_normal(size_img));     #This generates a complex noise signal.
    img_fft_noise = img_fft + noise # k-space
    img_noise = ifftn(ifftshift(img_fft_noise))# revert k-space back to noise
    return img_fft, img_fft_noise, img_noise

In [22]:
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {}
    train_and_val = ['train', 'val']
    data_path = [train_data_path, val_data_path]
    paths = {}
      
    for i in range(len(data_path)):

        data_list[train_and_val[i]] = []
        
        which_data_path = data_path[i]
      
    
        for fname in sorted(os.listdir(which_data_path + '/images')):
            if fname == '.DS_Store': continue
            
            subject_data_path = os.path.join(which_data_path + '/images', fname)
                     
            if not os.path.isfile(subject_data_path): continue 
            
     
            #get information from text file
            # this will return a tuple of root and extension
            split_tup = os.path.splitext(fname)

  
            # extract the file name and extension
            file_name = split_tup[0]
  
  
            data_list[train_and_val[i]].append((fname, subject_data_path))
          
            paths[train_and_val[i]].append((subject_data_path))
    
    return data_list, paths

In [33]:
os.getcwd()

'/Users/hoangvo/Desktop/homework/medicalImage/ML-reconstruction-Image/submit'

In [26]:
data_list, paths = load_data_path('data/train', 'data/val')

FileNotFoundError: [Errno 2] No such file or directory: 'data/train/images'

In [17]:

def custom_image_generator(files, data_instances, batch_size = 64):
    # to keep track that you don't have invalid index for number of files
    iter = 0
    
    while True:
        # check if you have a invalid index for files, if yes then reset it
        if (iter*batch_size + batch_size) > len(files):
            iter= 0
        end_iter = iter*batch_size + batch_size
        # Select files (paths/indices) for the batch
        paths = files[iter*batch_size:end_iter]
        # Read in each input and perform preprocessing (to batch of images)
        corrupted_images_batch = []
        original_images_batch = []
        for path in paths:
            print(path)
            if path == 'data/train/images/.DS_Store': continue
            img =  Image.open(path)
            original_images_batch.append(np.array(img)/255)
             
            im_fft, im_fft_noise, noise_im_frame = noise_and_kspace(img)
         # masked_img = draw_square_on_image(img)
            preprocess_img = np.array(noise_im_frame)
            corrupted_images_batch.append(preprocess_img/255)
        # Return a tuple of (corrupted_image_batch, true_image_batch) to feed the network
        corrupted_images_batch = np.array(corrupted_images_batch)
        original_images_batch = np.array(original_images_batch)
        # move to the next batch
        iter = iter + 1
    
        yield (corrupted_images_batch, original_images_batch)


In [18]:

## setting parameters
no_of_channels = 3
INPUT_DIM = (128,128,no_of_channels) # Image dimension
Z_DIM = [20, 100, 300, 5000]             # Dimension of the latent vector (z), model is trained using different values of Z_DIM

ae_encoder = None
ae_encoder_output = None
ae_encoder_input = Input(shape = INPUT_DIM, dtype=np.float32)

num_conv = {0: {'filters': 128, 'kernel_size': 5, 'strides': 2},
            1: {'filters': 64, 'kernel_size': 3, 'strides': 2},
            2: {'filters': 32, 'kernel_size': 3, 'strides': 2}}

encoder = ae_encoder_input
for layer_num,layer_data in num_conv.items():
  encoder = Conv2D(layer_data['filters'], layer_data['kernel_size'], layer_data['strides'], padding='same')(encoder)
  encoder = MaxPool2D((2,2),padding='same')(encoder)
  encoder = LeakyReLU(alpha=0.2)(encoder)
  encoder = BatchNormalization(axis=-1)(encoder)

volumeSize = K.int_shape(encoder)
flatten = Flatten()(encoder)
ae_encoder_output = Dense(Z_DIM[2])(flatten)
ae_encoder = Model(ae_encoder_input,ae_encoder_output)

ae_encoder.summary()
tf.keras.utils.plot_model(ae_encoder, show_shapes=True)


Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 128)       9728      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 32, 32, 128)       512       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 64)        73792     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 64)          0   

In [19]:
ae_decoder = None
ae_decoder_output = None
ae_decode_input = Input(shape=(Z_DIM[2],))
decoder = Dense(np.prod(volumeSize[1:]))(ae_decode_input)
decoder = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(decoder)

for layer_num,layer_data in reversed(sorted(num_conv.items())):
  decoder = Conv2DTranspose(layer_data['filters'], layer_data['kernel_size'], layer_data['strides'], padding='same')(decoder)
  decoder = UpSampling2D(size=(2,2))(decoder)
  decoder = LeakyReLU(alpha=0.2)(decoder)
  decoder = BatchNormalization(axis=-1)(decoder)

decoder = Conv2DTranspose(no_of_channels, 3, padding='same')(decoder)
ae_decoder_output = Activation('sigmoid')(decoder)
ae_decoder = Model(ae_decode_input, ae_decoder_output)

ae_decoder.summary()
tf.keras.utils.plot_model(ae_decoder, show_shapes=True)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 300)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               38528     
_________________________________________________________________
reshape_1 (Reshape)          (None, 2, 2, 32)          0         
_________________________________________________________________
conv2d_transpose_4 (Conv2DTr (None, 4, 4, 32)          9248      
_________________________________________________________________
up_sampling2d_3 (UpSampling2 (None, 8, 8, 32)          0         
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 8, 8, 32)          0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 8, 8, 32)          128 

In [20]:
autoencoder_model = None
# The input of the autoencoder will be the same as of encoder
autoencoder_input = ae_encoder_input
# The output of the autoencoder will be the output of decoder, when passed encoder input
autoencoder_output =  ae_decoder(ae_encoder(ae_encoder_input))
# Input to the combined model will be the input to the encoder.
# Output of the combined model will be the output of the decoder.
autoencoder_model = Model(autoencoder_input, autoencoder_output)
autoencoder_model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
model_3 (Functional)         (None, 300)               141580    
_________________________________________________________________
model_4 (Functional)         (None, 128, 128, 3)       275555    
Total params: 417,135
Trainable params: 416,239
Non-trainable params: 896
_________________________________________________________________


In [21]:
filenames = paths['train']
NUM_IMAGES = 2790
train_generator = custom_image_generator(filenames, NUM_IMAGES, batch_size) ## CREATE CUSTOM GENERATOR
optim.learning_rate = 0.01
autoencoder_model.compile(loss='mse', optimizer=Adam(), metrics='accuracy') ## MODEL COMPILING
autoencoder_model.fit(train_generator, steps_per_epoch=NUM_IMAGES /64, epochs=15)



d


FileNotFoundError: [Errno 2] No such file or directory: 'd'

In [10]:
NUM_IMAGES = 2790
BATCH_SIZE =  [64] #batch of images returned by image generator
N_EPOCHS = [15]    #epochs
LEARNING_RATE = [0.01,  0.001]  #learning rate
OPTIMIZERS = [Adam(),SGD()]
LOSS = ['mse']

filenames = 'data/train/images'
#data_path_val = 'data/val/images'
#data_list = load_data_path(data_path_train, data_path_val)

model = 0
save_model_paths = []
for batch_size in BATCH_SIZE:
  for epoch in N_EPOCHS:
    for learning_rate in LEARNING_RATE:
      for optimizer in OPTIMIZERS:
        for k_loss in LOSS:
          optim = optimizer
          print('epoch : ', epoch, ' lr : ', learning_rate, ' batch size : ', batch_size, ' optim : ', optim.get_config()['name'], ' loss: ', k_loss)
          train_generator = custom_image_generator(filenames, NUM_IMAGES, batch_size) ## CREATE CUSTOM GENERATOR
          optim.learning_rate = learning_rate
          autoencoder_model.compile(loss=k_loss, optimizer=optim, metrics='accuracy') ## MODEL COMPILING
          autoencoder_model.fit(train_generator, steps_per_epoch=NUM_IMAGES /batch_size, epochs=epoch)

          ## RECONSTRUCTION
          test_gen = custom_image_generator(filenames, NUM_IMAGES, batch_size)
          test_batch = next(test_gen)[0]
          test_images = test_batch[:10]
          reconst_images = autoencoder_model.predict(test_images)
          display_image_grid(test_images, 2, 5, "Incomplete Images")
          display_image_grid(reconst_images, 2, 5, "Reconstructed Images with Autoencoder")

        ## save model
          model_to_json = autoencoder_model.to_json()
          path_to_save = str(model)+'_epoch : '+ str(epoch)+ ' learning_rate : '+str(learning_rate) + ' optimizer : ' + optim.get_config()['name']+ ' batch_size : '+ str(batch_size) + ' k_loss : '+ k_loss + '.h5' 
          autoencoder_model.save(path_to_save)
          save_model_paths.append(path_to_save)
          model += 1 ## count no-of models


epoch :  15  lr :  0.01  batch size :  64  optim :  Adam  loss:  mse


FileNotFoundError: [Errno 2] No such file or directory: 'd'